In [101]:
from StoredQueries import POS_transactions
import pandas as pd

In [102]:
POS_df = POS_transactions()

In [103]:
# Create list of all transactions grouped by each transaction
POS_df.head()

,PartRno,PartId,PartName,LineItemID,TransactionID,Seq,PartRno,PartPriceRno,Qty,PartMeasureQty,UnitPrice,ExtPrice,CreDtTm
0,202,P70831298,TAIL GATE,3,2,1,202,202.0,1,1.000,29.9900,29.9900,2021-03-19 09:51:01.643
1,122,I70831252,SEAT BUCKET MANUAL,4,3,1,122,122.0,2,1.000,15.9900,31.9800,2021-03-19 10:02:09.660
2,75,E70831154,COIL,5,4,1,75,75.0,1,1.000,4.9900,4.9900,2021-03-19 10:07:59.860
3,125,I70831255,SEAT BELT (EACH),6,5,1,125,125.0,2,1.000,5.9900,11.9800,2021-03-19 11:04:12.900
4,10,A70831176,CAM SHAFT,7,6,1,10,10.0,1,1.000,15.9900,15.9900,2021-03-19 11:05:31.170


In [104]:
# Grouping the data by 'TransactionID' and aggregating the 'PartName' into a list
grouped_df = POS_df.groupby('TransactionID')['PartName'].apply(list).reset_index()

# Renaming the columns
grouped_df.columns = ['TransactionID', 'PartNames']



In [105]:
grouped_df.head(20)

,TransactionID,PartNames
0,2,[TAIL GATE]
1,3,[SEAT BUCKET MANUAL]
2,4,[COIL]
3,5,[SEAT BELT (EACH)]
4,6,[CAM SHAFT]
5,7,"[CONSOLE LID, CARPET]"
6,8,[RACK AND PINION MANUAL]
7,9,[COIL]
8,10,[WATER PUMP]
9,11,"[THROTTLE BODY, COIL, STARTER]"


In [106]:
# Sorting each list of part names and converting to a tuple
grouped_df['SortedPartNames'] = grouped_df['PartNames'].apply(lambda x: tuple(sorted(x)))

# Group by the sorted part names and count the occurrences
count_df = grouped_df.groupby('SortedPartNames').size().reset_index(name='Count')

# Renaming the columns for clarity
count_df.columns = ['TransactionLists', 'Count']

In [107]:
filtered_count_df = count_df[count_df['TransactionLists'].apply(lambda x: len(x) >= 2)]

In [108]:
filtered_count_df.nlargest(500, 'Count').to_csv('POS_Transactions_Count.csv', index=False)

This section performs the analysis of how often each item appears in a transaction with each other item.

In [109]:
from itertools import combinations
import pandas as pd
transactions = grouped_df["PartNames"]
unique_parts = set(part for sublist in transactions for part in sublist)

combs = list(combinations(unique_parts, 2))

comb_counts = {comb: 0 for comb in combs}

for transaction in transactions:
    for comb in combinations(transaction, 2):
        if comb in comb_counts:
            comb_counts[comb] += 1
        elif (comb[1], comb[0]) in comb_counts:
            comb_counts[(comb[1], comb[0])] += 1
            
comb_df = pd.DataFrame(list(comb_counts.items()), columns=['Combination', 'Count'])


In [110]:
comb_df['Count'] = comb_df['Count'].astype(int) # make sure these are integers, so they can be properly sorted
comb_df = comb_df.sort_values(by="Count", ascending=False).reset_index(drop=True)
comb_df.to_csv('POS_Transactions_Combinations.csv', index=False)

Create the second data frame from the combination dataframe. Step 1: Search through all transactions and create a items sold dataframe of the form (PartName, Qty, TotalPrice). Step 2: Loop through the combo data frame and create copies of each combination as A/B and one as B/A. Step 3: Match counts of the items sold dataframe to the combo dataframe to produce a ratio for each A/B and B/A record.

In [111]:
# Step 1: Search through all transactions and create a items sold dataframe of the form (PartName, Qty, TotalPrice).
part_count_df = POS_transactions()
part_count_df.head()

,PartRno,PartId,PartName,LineItemID,TransactionID,Seq,PartRno,PartPriceRno,Qty,PartMeasureQty,UnitPrice,ExtPrice,CreDtTm
0,202,P70831298,TAIL GATE,3,2,1,202,202.0,1,1.000,29.9900,29.9900,2021-03-19 09:51:01.643
1,122,I70831252,SEAT BUCKET MANUAL,4,3,1,122,122.0,2,1.000,15.9900,31.9800,2021-03-19 10:02:09.660
2,75,E70831154,COIL,5,4,1,75,75.0,1,1.000,4.9900,4.9900,2021-03-19 10:07:59.860
3,125,I70831255,SEAT BELT (EACH),6,5,1,125,125.0,2,1.000,5.9900,11.9800,2021-03-19 11:04:12.900
4,10,A70831176,CAM SHAFT,7,6,1,10,10.0,1,1.000,15.9900,15.9900,2021-03-19 11:05:31.170


In [112]:
# Step 1: Initial setup
part_count_df = part_count_df[["PartName", "Qty", "UnitPrice"]]
part_count_df['TotalPrice'] = (part_count_df['Qty'] * part_count_df['UnitPrice'])
part_count_df['TotalPrice'] = part_count_df['TotalPrice'].astype(float).round(2)

# Step 2: Count the number of times each PartName appears
part_count_series = part_count_df['PartName'].value_counts().reset_index()

# Rename the columns to make it clear
part_count_series.columns = ['PartName', 'PartCount']

# Step 3: Sum Part Quantity by PartName
total_qty_series = part_count_df.groupby('PartName')['Qty'].sum().reset_index()
total_qty_series.columns = ['PartName', 'TotalPartQuantity']

# Step 4: Merge the TotalPartQuantity and PartCount back with the original part_count_df
part_count_df = pd.merge(part_count_df, part_count_series, on='PartName', how='left')
part_count_df = pd.merge(part_count_df, total_qty_series, on='PartName', how='left')

In [113]:
part_count_df.head(10)

,PartName,Qty,UnitPrice,TotalPrice,PartCount,TotalPartQuantity
0,TAIL GATE,1,29.9900,29.99,197,218
1,SEAT BUCKET MANUAL,2,15.9900,31.98,873,1394
2,COIL,1,4.9900,4.99,588,1243
3,SEAT BELT (EACH),2,5.9900,11.98,177,391
4,CAM SHAFT,1,15.9900,15.99,87,106
5,CONSOLE LID,1,3.9900,3.99,356,392
6,CARPET,1,5.9900,5.99,248,290
7,RACK AND PINION MANUAL,1,29.9900,29.99,352,372
8,COIL,1,4.9900,4.99,588,1243
9,WATER PUMP,1,17.9900,17.99,171,171


In [114]:
grouped_df = part_count_df.groupby('PartName').agg({'TotalPrice': 'sum'}).reset_index()

# Add the PartCount feature that counts the number of times each item appears in a transaction
grouped_df = pd.merge(grouped_df, part_count_series, on='PartName', how='left')

# Add the TotalPartQuantity feature from part_count_df, which measures the total number of each item sold across all transactions.
grouped_df = pd.merge(grouped_df, total_qty_series, on='PartName', how='left')

In [115]:
grouped_df.head()

,PartName,TotalPrice,PartCount,TotalPartQuantity
0,A FRAME,149.94,5,6
1,A/C COMPRESSOR,28717.21,1021,1062
2,A/C CONDENSER,13661.75,615,625
3,A/C EVAPORATOR,286.79,21,21
4,A/C REGULATOR,7.99,1,1


In [116]:
# Step 2: Loop through the combo data frame and create copies of each combination as A/B and one as B/A.
def create_combination_pairs(row):
    part1, part2 = row['Combination']
    return pd.DataFrame({
        'Part1': [part1, part2],
        'Part2': [part2, part1],
        'Count': [row['Count'], row['Count']]
    })

# Apply the function to each row and concatenate the results
expanded_df = pd.concat(comb_df.apply(create_combination_pairs, axis=1).tolist()).reset_index(drop=True)


In [117]:
expanded_df.head()

,Part1,Part2,Count
0,WHEEL STEEL,TIRE 13-16,462
1,TIRE 13-16,WHEEL STEEL,462
2,WINDOW REGULATOR,WINDOW MOTOR,416
3,WINDOW MOTOR,WINDOW REGULATOR,416
4,FENDER CAR,HOOD,397


In [118]:
# Step 3: Match counts of the items sold dataframe to the combo dataframe to produce a ratio for each A/B and B/A record.
# First merge on Part1
merged_df_part1 = pd.merge(expanded_df, grouped_df, left_on='Part1', right_on='PartName', how='left')
merged_df_part1 = merged_df_part1.rename(columns={'Qty': 'Qty_Part1', 'TotalPrice': 'TotalPrice_Part1'})

# Second merge on Part2
final_merged_df = pd.merge(merged_df_part1, grouped_df, left_on='Part2', right_on='PartName', how='left')
final_merged_df = final_merged_df.rename(columns={'Qty': 'Qty_Part2', 'TotalPrice': 'TotalPrice_Part2'})

# Remove extra columns
final_merged_df = final_merged_df.drop(columns=['PartName_x', 'PartName_y'], axis='columns')

# Include part ratio
final_merged_df['PartSaleRatio'] = (final_merged_df['Count'] / final_merged_df['PartCount_x']).round(2)

# Include Average part price
final_merged_df['TotalPrice_Part1'] = pd.to_numeric(final_merged_df['TotalPrice_Part1'], errors='coerce')
final_merged_df['PartCount_x'] = pd.to_numeric(final_merged_df['TotalPartQuantity_x'], errors='coerce')
final_merged_df['Part1_Average_Price'] = (final_merged_df['TotalPrice_Part1'] / final_merged_df['PartCount_x']).round(2)

# Display Head
final_merged_df.head(25)


,Part1,Part2,Count,TotalPrice_Part1,PartCount_x,TotalPartQuantity_x,TotalPrice_Part2,PartCount_y,TotalPartQuantity_y,PartSaleRatio,Part1_Average_Price
0,WHEEL STEEL,TIRE 13-16,462,7007.38,1052,1052,73497.13,1659,3983,0.70,6.66
1,TIRE 13-16,WHEEL STEEL,462,73497.13,3983,3983,7007.38,656,1052,0.28,18.45
2,WINDOW REGULATOR,WINDOW MOTOR,416,8204.72,783,783,8200.41,608,709,0.62,10.48
3,WINDOW MOTOR,WINDOW REGULATOR,416,8200.41,709,709,8204.72,676,783,0.68,11.57
4,FENDER CAR,HOOD,397,50619.35,1915,1915,46661.84,1281,1336,0.24,26.43
5,HOOD,FENDER CAR,397,46661.84,1336,1336,50619.35,1639,1915,0.31,34.93
6,HEADLAMP COMPOSITE,BUMPER STEEL,393,60749.99,2651,2651,68959.94,1911,2017,0.22,22.92
7,BUMPER STEEL,HEADLAMP COMPOSITE,393,68959.94,2017,2017,60749.99,1821,2651,0.21,34.19
8,FENDER CAR,HEADLAMP COMPOSITE,384,50619.35,1915,1915,60749.99,1821,2651,0.23,26.43
9,HEADLAMP COMPOSITE,FENDER CAR,384,60749.99,2651,2651,50619.35,1639,1915,0.21,22.92


In [119]:
final_merged_df.to_csv('POS_Transactions_Analysis.csv', index=False)